# Import Dependencies

In [1]:
!pip install gym gym-retro

     -------------------------------------- 721.7/721.7 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     -------------------------------------- 152.0/152.0 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 14.9/14.9 MB 5.1 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 2.3 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827646 sha256=e01f07fc7a7deb3d127349ec5d9a337329bb9075b0c5cbc13aeb9d1da3877146
  Stored in directory: c:\users\mason\appdata\local\pip\cache\wheels\17\79\65\7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym


# Setup SF2 environment

In [ ]:
import retro

